In [1]:
import os
from pathlib import Path

import numpy as np
import cv2
import supervision as sv
from autodistill.detection import CaptionOntology
from autodistill_grounded_sam import GroundedSAM
# from autodistill_yolov8 import YOLOv8
from ultralytics import YOLO
from IPython.display import Image
import matplotlib.pyplot as plt
from boxmot import (OCSORT, BoTSORT, BYTETracker, DeepOCSORT, StrongSORT,
                    create_tracker, get_tracker_config)

import video_multiprocessing as vm
import superimpose_boxes as sb

HOME = os.getcwd()
HOME

CUDA not available. Using MPS.
CUDA not available. Using MPS.


'/Users/eric/Desktop/2-Career/Projects/ObjectDetection'

In [2]:
# latest_train_dir = input("Enter the name of the latest training directory: ")
latest_train_dir = 'train_2023.12.03_19.33.40_n_im640_ep120_baNone_seNone'
weights = 'best' 
# # weights = 'last' 
model = YOLO(f"runs/detect/{latest_train_dir}/weights/{weights}.pt")
!tree {HOME}/runs/detect/{latest_train_dir}/

/Users/eric/Desktop/2-Career/Projects/ObjectDetection/runs/detect/train_2023.12.03_19.33.40_n_im640_ep120_baNone_seNone/
├── args.yaml
├── labels.jpg
├── labels_correlogram.jpg
├── results.csv
├── train_batch0.jpg
├── train_batch1.jpg
├── train_batch2.jpg
└── weights
    ├── best.pt
    └── last.pt

2 directories, 9 files


### BoT-SORT Parameters
<b>track_high_thresh</b>: A confidence threshold to determine if a detection should initially be considered for tracking. 
A higher threshold means only detections with high confidence are tracked, potentially reducing false positives but possibly missing less confident true objects.
<br>
<b>track_low_thresh</b>: Once an detection is being tracked, this threshold is used in subsequent frames to decide if a detection continues to match with the existing track. A lower threshold is more lenient, which can be useful in maintaining track of objects even when the model's confidence in them dips temporarily.
<br>
<b>new_track_thresh</b>: This determines how confident the model should be to start tracking (a new detection / a detection that does not match any existing tracks).
<br>
<b>track_buffer</b>: This is the number of frames an object can be missing or not detected before its track is terminated.
<br>
<b>match_thresh</b>: This is the threshold for matching detections to the existing tracks between frames. A higher threshold requires detections to be more similar to the existing track, reducing identity switches but potentially losing track more often.
<br>
<b>gmc_method</b>: The method used to calculate the IOU of two bounding boxes. Can be either 'iou' or 'giou'.
make a bullet list of the items below
<ul>
<li> <b>sparseOptFlow</b>: This setting, standing for Global Motion Compensation, is specific to BoT-SORT.</li>
<li> <b>orb</b>: ORB is a fast feature detector and descriptor. It is used for detecting keypoints and computing descriptors that can be matched across frames.
ORB is useful for tracking objects in scenarios where you need a balance between speed and accuracy. It's less sensitive to changes in lighting compared to some other methods.</li>
<li> <b>sift</b>: SIFT is a more computationally intensive method compared to ORB, used for detecting and describing local features in images.
SIFT is highly robust against changes in scale, noise, and illumination, making it suitable for applications where high accuracy is required, even if it means slower processing.</li>
<li> <b>ecc</b>: ECC is an algorithm for aligning images (finding the optimal spatial transformation between them). It's particularly good at handling small motions and subtle transformations.
ECC can be used for precise alignment of video frames, which is useful in situations where the camera is moving slightly or where very accurate tracking of objects is required.</li>

In [ ]:
# from datetime import datetime

# botsort_contents = '''# BoT-SORT tracker https://github.com/NirAharon/BoT-SORT

# # tracker_type: This specifies the type of tracker being used. In your case, 'bytetrack' is selected. 
# # ByteTrack is known for its efficiency and accuracy in tracking multiple objects even with occlusions or in crowded scenes.

# # track_high_thresh: This is the high threshold for the first association in tracking. 
# # It's a confidence threshold used to determine if an object detected by the YOLO model should initially be considered for tracking. 
# # A higher threshold means only detections with high confidence are tracked, potentially reducing false positives but possibly missing less confident true objects.

# # track_low_thresh: This is the lower threshold for the second association. 
# # Once an object is being tracked, this threshold is used in subsequent frames to decide if a detected object continues to match with the existing track. 
# # A lower threshold here allows for more lenient tracking, which can be useful in maintaining track of objects even when the model's confidence in them dips temporarily.

# # new_track_thresh: This threshold is used to initiate a new track if a detection does not match any existing tracks. 
# # It determines how confident the model should be to start tracking a new object. 
# # A higher value means fewer new tracks will be created, reducing clutter but possibly missing real objects.

# # track_buffer: This setting determines the buffer time to calculate when to remove tracks. 
# # It's essentially the number of frames an object can be missing or not detected before its track is terminated. 
# # A larger buffer allows tracks to be maintained even when objects are occluded or miss detection for a few frames.

# # match_thresh: This is the threshold for matching tracks between frames. 
# # It's used in the data association step to decide whether a detected 
# # object in the current frame matches an existing track. 
# # A higher threshold ensures that only detections very similar to the existing track 
# # are matched, reducing identity switches but potentially losing track more often.

# # gmc_method: This setting, standing for Global Motion Compensation, is specific to BoT-SORT. 
#     # sparseOptFlow: method compensates for camera motion by using optical flow techniques, 
#     # helping maintain track stability even with moving cameras.

#     # orb: ORB is a fast feature detector and descriptor. It is used for detecting keypoints and computing 
#     # descriptors that can be matched across frames.
#     # ORB is useful for tracking objects in scenarios where you need a balance between speed and accuracy. 
#     # It's less sensitive to changes in lighting compared to some other methods.

#     # sift: SIFT is a more computationally intensive method compared to ORB, used for detecting and describing local features in images.
#     # SIFT is highly robust against changes in scale, noise, and illumination, making it suitable for applications where 
#     # high accuracy is required, even if it means slower processing.

#     # ecc: ECC is an algorithm for aligning images (finding the optimal spatial transformation between them). 
#     # It's particularly good at handling small motions and subtle transformations.
#     # ECC can be used for precise alignment of video frames, which is useful in situations where 
#     # the camera is moving slightly or where very accurate tracking of objects is required.

# # proximity_thresh: A threshold for proximity-based matching in tracking. 
# # It likely influences how close in terms of spatial distance a detected object needs to be across frames to be considered the same object.
# # For instance, if the distance metric is based on the relative positions of bounding boxes in a normalized coordinate space 
# # (like a frame of 1080p video being scaled to a 1x1 square), 
# # 0.5 could mean that the centers of the bounding boxes are at most half the width/height of the normalized space apart.

# # appearance_thresh: This threshold might be related to appearance-based matching, determining how similar in appearance 
# # a detected object needs to be across frames to maintain the same track. 
# # This threshold is a normalized value, where 0.0 would represent no requirement for appearance similarity (any object could be matched with any other), 
# # and 1.0 would represent a requirement for exact appearance matching. 
# # Thus, a value of 0.25 indicates a relatively lenient requirement for appearance similarity. 
# # It allows for some variation in how an object appears across frames, which can be useful in dynamic environments where lighting, orientation, or partial occlusions 
# # might slightly alter an object's appearance.

# # with_reid: Indicates whether Re-identification (ReID) features are used. ReID involves using features based on the object’s appearance 
# # to re-identify and track objects across different camera views or after long periods of occlusion. False indicates it is not currently used.

# tracker_type: botsort  # tracker type, ['botsort', 'bytetrack']
# track_high_thresh: {track_high_thresh}  # threshold for the first association
# track_low_thresh: {track_low_thresh}  # threshold for the second association
# new_track_thresh: {new_track_thresh}  # threshold for init new track if the detection does not match any tracks
# track_buffer: {track_buffer}  # buffer to calculate the time when to remove tracks
# match_thresh: {match_thresh} # threshold for matching tracks
# # min_box_area: 10  # threshold for min box areas(for tracker evaluation, not used for now)
# # mot20: False  # for tracker evaluation(not used for now)

# # BoT-SORT settings
# gmc_method: {gmc_method}  # method of global motion compensation [sparseOptFlow, orb, sift, ecc]
# # ReID model related thresh (not supported yet)
# proximity_thresh: {proximity_thresh}
# appearance_thresh: {appearance_thresh}
# with_reid: {with_reid}'''
# botsort_args = {
#     'track_high_thresh': 0.5,  # determines if detection should initially be considered for tracking
#     'track_low_thresh': 0.01,  # determines if a detected object continues to match with the existing track
#     'new_track_thresh': 0.60,  # determines how confident the model should be to start tracking a new object
#     'track_buffer': 99999,  # number of frames an object can be missing or not detected before its track is terminated
#     'match_thresh': 0.99, # determines whether a detected object in the current frame matches an existing track
#     'gmc_method': 'orb',  # [sparseOptFlow, orb, sift, ecc] Global Motion Compensation Method
#     # Below are not supported yet
#     'proximity_thresh': 0.95,  # threshold for proximity-based matching in tracking
#     'appearance_thresh': 0.91,  # threshold for appearance-based matching in tracking
#     'with_reid': True,  # indicates whether Re-identification (ReID) features are used
# }
# botsort_contents = botsort_contents.format(**botsort_args)
# name = f"Track {botsort_args['track_low_thresh']}<T<{botsort_args['track_high_thresh']} New_{botsort_args['new_track_thresh']} Match_{botsort_args['match_thresh']} GMC_{botsort_args['gmc_method']}"
# videos = {
#     1: 'videos/58.mp4',
#     2: 'images',
#     3: 'hockey/videos/STL_VS_SAJ_003.mp4',
#     4: '002.mp4',
#     5: '002_trim.mp4',
#     6: '002_trim2.mp4',
#     7: 'hockey/videos/CHI_VS_TOR_004.mp4',
# }
# yolo_args = {
#     'source': videos[6],
#     'device': 'mps',
#     'tracker': 'botsort1.yaml',
#     'conf': 0.25,
#     'iou': 0.3,
#     'save': True,
#     'save_txt': False,
#     'save_conf': True,
#     'line_thickness': 1,
#     'imgsz': 640,
#     'name': name,
#     # 'retina_masks': True,
# }
# weights = 'best' 
# # weights = 'last' 

# # write to file botsort1.yaml
# with open('botsort1.yaml', 'w') as f:
#     f.write(botsort_contents)



# print(f'Using the {weights.upper()} weights from runs/detect/{latest_train_dir}\n')
# print(f'Saving to runs/detect/{latest_train_dir}/results\n')
# # model = YOLO(f"epoch7.pt")
# model = YOLO(f"runs/detect/{latest_train_dir}/weights/{weights}.pt")
# # results = model.predict(**yolo_args) 
# # results = model.track(**yolo_args) 

In [ ]:
break
'''Attempting to integrate Trackers without using Ultralytics below 
because Ultralytics is not compatible with REID'''

In [ ]:
def create_video_writer(video_cap, output_filename):

    # grab the width, height, and fps of the frames in the video stream.
    frame_width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video_cap.get(cv2.CAP_PROP_FPS))

    # initialize the FourCC and a video writer object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter(output_filename, fourcc, fps,
                             (frame_width, frame_height))

    return writer


device = "mps:0" # cuda:0 , cpu
fp16 = True # True if gpu available
source = "002.mp4"

class_dict = {
    # class, color (BGR)
    0: ('skater', (0, 30, 255)),
    1: ('goalie', (0, 255, 50)),
    2: ('referee', (255, 180, 0)),
}
tracker = DeepOCSORT(
    model_weights=Path('osnet_x0_25_msmt17.pt'), # which ReID model to use
    device=device,
    fp16=fp16,
)
vid = cv2.VideoCapture(source)
writer = create_video_writer(vid, "DeepOCSORT.mp4")

while True:
    ret, im = vid.read()

    detections = model.predict(im )[0]

    # initialize the list of bounding boxes and confidences
    results = []
    if not ret:
        break

    if np.array(detections.boxes.data.tolist()).ndim < 2:
        results = [[0, 0, 0, 0, 0.0922948837280273, 0]] # --> (x, y, x, y, id, conf, cls)
    ts = tracker.update(np.array(detections.boxes.data.tolist()), im) # --> (x, y, x, y, id, conf, cls)

    xyxys = ts[:,0:4].astype('int') # float64 to int
    ids = ts[:, 4].astype('int') # float64 to int 
    confs = ts[:, 5]
    clss = ts[:, 6]

    # print bboxes with their associated id, cls and conf
    if ts.shape[0] != 0:
        for xyxy, id, conf, cls in zip(xyxys, ids, confs, clss):
            im = cv2.rectangle(
                im,
                (xyxy[0], xyxy[1]),
                (xyxy[2], xyxy[3]),
                class_dict[cls][1],
                1
            )
            cv2.putText(
                im,
                f'{class_dict[cls][0]}, confidence: {round(conf, 1)}, id:{id}',
                (xyxy[0], xyxy[1]-10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                class_dict[cls][1],
                1
            )

    writer.write(im)


vid.release()
writer.release()

In [ ]:
# import cv2
# from boxmot import BoTSORT, DeepOCSORT, OCSORT, HybridSORT, StrongSORT
# from pathlib import Path

# # Initialize your YOLO object detection model
# model = YOLO(f"runs/detect/{latest_train_dir}/weights/{weights}.pt")

# # Initialize the DeepOCSORT tracker
# tracker = BoTSORT(
#     model_weights=Path('osnet_x0_25_msmt17.pt'), # ReID model
#     device='mps',
#     fp16=True,
#     # img=None,
# )
# yolo_args_frame = {
#     'device': 'mps',
#     'tracker': 'botsort1.yaml',
#     'conf': 0.25,
#     'iou': 0.3,
#     'save': True,
#     'save_txt': False,
#     'save_conf': True,
#     'line_thickness': 1,
#     'imgsz': 640,
#     'name': name,
#     # 'retina_masks': True,
# }

# video_path = yolo_args['source']
# vid = cv2.VideoCapture(video_path)
# # vid = cv2.VideoCapture(0)
# color = (0, 0, 255)  # BGR
# thickness = 2
# fontscale = 0.5

# def process_detections(results):
#     detections = []
#     for r in results:
#         for xyxy, confidence, classification in zip(r.boxes.xyxy.cpu(), r.boxes.conf.cpu(), r.boxes.cls.cpu()):
#             # Convert tensor to numpy array and round off
#             bbox = xyxy.numpy().astype(int).tolist()
#             detections.append((*bbox, confidence.item(), classification.item()))
#     # Convert detections to a numpy array
#     return np.array(detections)

# while True:
#     ret, im = vid.read()

#     results = model.predict(source=im, **yolo_args_frame) 
#     dets = process_detections(results)

#     try:
#         tracks = tracker.update(dets, im) # --> (x, y, x, y, id, conf, cls, ind)
#     except:
#         # print(im.shape)
#         continue
#         # vid.release()
#         # cv2.destroyAllWindows()

#     xyxys = tracks[:, 0:4].astype('int') # float64 to int
#     ids = tracks[:, 4].astype('int') # float64 to int
#     confs = tracks[:, 5]
#     clss = tracks[:, 6].astype('int') # float64 to int
#     inds = tracks[:, 7].astype('int') # float64 to int

#     # print bboxes with their associated id, cls and conf
#     if tracks.shape[0] != 0:
#         for xyxy, id, conf, cls in zip(xyxys, ids, confs, clss):
#             im = cv2.rectangle(
#                 im,
#                 (xyxy[0], xyxy[1]),
#                 (xyxy[2], xyxy[3]),
#                 color,
#                 thickness
#             )
#             cv2.putText(
#                 im,
#                 f'id: {id}, conf: {conf}, c: {cls}',
#                 (xyxy[0], xyxy[1]-10),
#                 cv2.FONT_HERSHEY_SIMPLEX,
#                 fontscale,
#                 color,
#                 thickness
#             )

#     # show image with bboxes, ids, classes and confidences
#     cv2.imshow('frame', im)

#     # break on pressing q
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# vid.release()
# cv2.destroyAllWindows()

In [ ]:
# import cv2
# from boxmot import BoTSORT, DeepOCSORT, OCSORT, HybridSORT, StrongSORT
# from pathlib import Path

# # Initialize your YOLO object detection model
# model = YOLO(f"runs/detect/{latest_train_dir}/weights/{weights}.pt")

# # Initialize the DeepOCSORT tracker
# tracker = BoTSORT(
#     model_weights=Path('osnet_x0_25_msmt17.pt'), # ReID model
#     device='mps',
#     fp16=True,
#     # img=None,
# )
# yolo_args_frame = {
#     'device': 'mps',
#     'tracker': 'botsort1.yaml',
#     'conf': 0.25,
#     'iou': 0.3,
#     'save': True,
#     'save_txt': False,
#     'save_conf': True,
#     'line_thickness': 1,
#     'imgsz': 640,
#     'name': name,
#     # 'retina_masks': True,
# }

# def process_detections(results):
#     detections = []
#     for r in results:
#         for xyxy, confidence, classification in zip(r.boxes.xyxy.cpu(), r.boxes.conf.cpu(), r.boxes.cls.cpu()):
#             # Convert tensor to numpy array and round off
#             bbox = xyxy.numpy().astype(int).tolist()
#             detections.append((*bbox, confidence.item(), classification.item()))
#     # Convert detections to a numpy array
#     return np.array(detections)


# def main(video_path, model, tracker):
#     # Open the video
#     cap = cv2.VideoCapture(video_path)

#     if not cap.isOpened():
#         print("Error: Could not open video.")
#         return

#     # Get video properties for the output file
#     frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
#     frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
#     fps = cap.get(cv2.CAP_PROP_FPS)

#     # Define the codec and create VideoWriter object
#     fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or 'XVID' if 'mp4v' does not work
#     out = cv2.VideoWriter('output_video.mp4', fourcc, fps, (frame_width, frame_height))

#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             break

#         # Process frame with YOLO model to get results
#         results = model.predict(source=frame, **yolo_args_frame) 

#         # Process detections
#         detections = process_detections(results)

#         try:

#             # Update the tracker with the new detections
#             tracked_objects = tracker.update(detections, frame)

#         # Draw tracked objects on the frame
#             for obj in tracked_objects:
#                 # Draw a rectangle around the object
#                 # print(obj, obj[0], obj[1], obj[2], obj[3], type(obj[0]), type(obj[1]), type(obj[2]), type(obj[3]))
#                 cv2.rectangle(frame, (int(obj[0]), int(obj[1])), (int(obj[2]), int(obj[3])), (0, 255, 0), 2)
                
#                 # Draw the object ID near the object
#                 cv2.putText(frame, str(obj[4]), (int(obj[0]), int(obj[1]) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

#             out.write(frame)
#         except:
#             pass
#     # Release everything when job is finished
#     cap.release()
#     out.release()

# if __name__ == "__main__":
#     video_path = yolo_args['source']
#     main(video_path, model, tracker)
